In [0]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots
import warnings
warnings.filterwarnings('ignore')

# =============================================================================
# LOAD DATA (from Kaggle - already cleaned)
# =============================================================================

df = pd.read_csv('/Volumes/workspace/default/nyc-housing/nyc_housing_base.csv')

print("="*80)
print("📂 KAGGLE DATASET LOADED (Pre-Cleaned)")
print("="*80)
print(f"Total records: {len(df):,}")
print(f"Source: NYC Housing Prices (Kaggle)")
print(f"Columns: {list(df.columns)}\n")

# =============================================================================
# MINIMAL PROCESSING (Convert types & add boroughs)
# =============================================================================

BOROUGH_NAMES = {
    1: 'Manhattan',
    2: 'Bronx',
    3: 'Brooklyn',
    4: 'Queens',
    5: 'Staten Island'
}

BOROUGH_COLORS = {
    'Manhattan': '#E63946',
    'Brooklyn': '#457B9D',
    'Queens': '#2A9D8F',
    'Bronx': '#9B59B6',
    'Staten Island': '#F4A261'
}

def assign_borough_from_coordinates(lat, lon):
    """Assign borough based on coordinates."""
    if pd.isna(lat) or pd.isna(lon):
        return 'Unknown'
    if lon < -74.05:
        return 'Staten Island'
    if lat >= 40.79 and lon > -74.05:
        return 'Bronx'
    if 40.70 <= lat <= 40.88 and -74.02 <= lon <= -73.93:
        return 'Manhattan'
    if lat < 40.70 and lon > -74.05 and lon < -73.85:
        return 'Brooklyn'
    if lat < 40.74 and lon <= -73.92:
        return 'Brooklyn'
    if lon >= -73.88:
        return 'Queens'
    if lat < 40.74:
        return 'Brooklyn'
    return 'Queens'

print("\n🔧 Processing data...")

# Convert numeric columns
numeric_cols = ['sale_price', 'bldgarea', 'lotarea', 'resarea', 'comarea',
                'yearbuilt', 'building_age', 'numfloors', 'unitsres', 
                'unitstotal', 'latitude', 'longitude']

for col in numeric_cols:
    if col in df.columns:
        df[col] = pd.to_numeric(df[col], errors='coerce')

# Remove only impossible values (0 or negative building area)
initial_count = len(df)
df = df[~((df['bldgarea'].notna()) & (df['bldgarea'] <= 0))].copy()
print(f"✓ Removed {initial_count - len(df)} records with invalid building area")

# Assign boroughs
has_coords = df['latitude'].notna() & df['longitude'].notna()
df.loc[has_coords, 'borough_name'] = df[has_coords].apply(
    lambda row: assign_borough_from_coordinates(row['latitude'], row['longitude']), axis=1
)

# Fallback to borough_x
if 'borough_x' in df.columns:
    no_coords = ~has_coords & df['borough_name'].isna()
    df.loc[no_coords, 'borough_name'] = df.loc[no_coords, 'borough_x'].map(BOROUGH_NAMES)

df['borough_name'] = df['borough_name'].fillna('Unknown')

# =============================================================================
# CALCULATE METRICS & CLASSIFY PROPERTIES
# =============================================================================

# Price per square foot
has_area = df['bldgarea'] > 0
df.loc[has_area, 'price_per_sqft'] = df.loc[has_area, 'sale_price'] / df.loc[has_area, 'bldgarea']

# Classify by typical residential vs commercial/special patterns
def classify_property(row):
    """Classify property based on characteristics."""
    price = row['sale_price']
    area = row['bldgarea']
    pps = row['price_per_sqft']
    
    # Mega commercial deals
    if pd.notna(area) and area > 500000:  # >500K sqft
        return 'Large Commercial/Portfolio'
    
    # Air rights/land deals (very low price per sqft)
    if pd.notna(pps) and pps < 10:
        return 'Air Rights/Land/Special'
    
    # Luxury residential
    if pd.notna(pps) and pps > 2000:
        return 'Luxury Residential'
    
    # Standard residential
    if pd.notna(pps) and 100 <= pps <= 2000:
        return 'Standard Residential'
    
    # Budget residential
    if pd.notna(pps) and 10 <= pps < 100:
        return 'Budget/Outer Borough'
    
    return 'Other'

df['property_segment'] = df.apply(classify_property, axis=1)

# Price tiers
df['price_tier'] = pd.cut(
    df['sale_price'],
    bins=[0, 300000, 600000, 900000, 1500000, 3000000, float('inf')],
    labels=['Under $300K', '$300K-600K', '$600K-900K', '$900K-1.5M', 
            '$1.5M-3M', 'Over $3M']
)

# Size categories
df['size_category'] = pd.cut(
    df['bldgarea'],
    bins=[0, 800, 1500, 2500, 5000, 10000, float('inf')],
    labels=['Small (<800)', 'Medium (800-1.5K)', 'Large (1.5-2.5K)', 
            'Very Large (2.5-5K)', 'Massive (5-10K)', 'Commercial (10K+)']
)

# =============================================================================
# DATA SUMMARY
# =============================================================================

print("\n" + "="*80)
print("📊 DATA SUMMARY")
print("="*80)

print(f"\nTotal Properties: {len(df):,}")

print(f"\n💰 Price Statistics:")
print(f"  Range:  ${df['sale_price'].min():,.0f} - ${df['sale_price'].max():,.0f}")
print(f"  Median: ${df['sale_price'].median():,.0f}")
print(f"  Mean:   ${df['sale_price'].mean():,.0f}")

print(f"\n📏 Size Statistics:")
print(f"  Range:  {df['bldgarea'].min():,.0f} - {df['bldgarea'].max():,.0f} sqft")
print(f"  Median: {df['bldgarea'].median():,.0f} sqft")

print(f"\n💵 Price/SqFt (All Properties):")
pps_valid = df[df['price_per_sqft'].notna()]['price_per_sqft']
print(f"  Range:  ${pps_valid.min():,.2f} - ${pps_valid.max():,.0f}")
print(f"  Median: ${pps_valid.median():,.0f}")
print(f"  Mean:   ${pps_valid.mean():,.0f}")

# Show percentiles to understand distribution
print(f"\n  Percentiles:")
for p in [5, 25, 50, 75, 95]:
    val = pps_valid.quantile(p/100)
    print(f"    {p}th: ${val:,.0f}")

print(f"\n🏘️  Borough Distribution:")
for borough, count in df['borough_name'].value_counts().items():
    pct = count / len(df) * 100
    median_price = df[df['borough_name'] == borough]['sale_price'].median()
    print(f"  {borough:<20} {count:>8,} ({pct:>5.1f}%)  Median: ${median_price:>10,.0f}")

print(f"\n🏢 Property Segments:")
for segment, count in df['property_segment'].value_counts().items():
    pct = count / len(df) * 100
    subset = df[df['property_segment'] == segment]
    median_price = subset['sale_price'].median()
    median_pps = subset['price_per_sqft'].median()
    print(f"  {segment:<30} {count:>6,} ({pct:>5.1f}%)  Median: ${median_price:>10,.0f}  (${median_pps:>6.0f}/sqft)")

# Show the ultra-low price/sqft properties
print(f"\n⚠️  Ultra-Low Price/SqFt Properties (<$10/sqft):")
ultra_low = df[df['price_per_sqft'] < 10].sort_values('price_per_sqft')
print(f"  Count: {len(ultra_low)} properties")
if len(ultra_low) > 0:
    print(f"\n  {'Sale Price':<15} {'Area (sqft)':<15} {'$/sqft':<10} {'Borough':<15} {'Type'}")
    print("  " + "-"*75)
    for idx, row in ultra_low.head(10).iterrows():
        area_str = f"{row['bldgarea']:,.0f}" if pd.notna(row['bldgarea']) else "N/A"
        pps_str = f"${row['price_per_sqft']:,.2f}" if pd.notna(row['price_per_sqft']) else "N/A"
        bldg_type = row.get('bldgclass', 'N/A')
        print(f"  ${row['sale_price']:>12,.0f}  {area_str:>13}  {pps_str:<10} {row['borough_name']:<15} {bldg_type}")
    print(f"\n  → These are likely air rights, land transfers, or special commercial deals")

# =============================================================================
# CREATE FILTERED DATASETS
# =============================================================================

# Standard residential filter (most useful for typical housing analysis)
df_residential = df[
    (df['property_segment'].isin(['Standard Residential', 'Luxury Residential', 'Budget/Outer Borough'])) &
    (df['price_per_sqft'] >= 50)  # Reasonable floor for actual housing
].copy()

print(f"\n" + "="*80)
print("💾 SAVING DATASETS")
print("="*80)

# Save all data
df.to_csv('nyc_housing_complete.csv', index=False)
print(f"  ✓ nyc_housing_complete.csv ({len(df):,} records) - All properties")

# Save residential only
df_residential.to_csv('nyc_housing_residential.csv', index=False)
print(f"  ✓ nyc_housing_residential.csv ({len(df_residential):,} records) - Typical housing")

print(f"\n📊 Recommendations:")
print(f"  • Use 'nyc_housing_residential.csv' for typical housing price analysis")
print(f"  • Use 'nyc_housing_complete.csv' if you want to include commercial/special deals")
print(f"  • The ultra-low $/sqft properties are real, just unusual (air rights, bulk land, etc.)")

print(f"\n✅ Data ready for analysis!")

In [0]:

def verify_borough_assignment(df):
    """Create verification plot to check if borough assignment is correct."""
    
    print("\n" + "="*60)
    print("🗺️ VERIFYING BOROUGH ASSIGNMENT")
    print("="*60)
    
    fig, ax = plt.subplots(figsize=(14, 12), facecolor='#0D1117')
    ax.set_facecolor('#0D1117')
    
    # Plot each borough with different color
    for borough in ['Staten Island', 'Bronx', 'Queens', 'Brooklyn', 'Manhattan']:
        data = df[df['borough_name'] == borough]
        
        # Sample if too many points
        if len(data) > 1000:
            data = data.sample(1000, random_state=42)
        
        if len(data) > 0:
            ax.scatter(
                data['longitude'], 
                data['latitude'], 
                c=BOROUGH_COLORS[borough], 
                s=8, 
                alpha=0.6, 
                label=f"{borough} ({len(df[df['borough_name']==borough]):,})",
                edgecolors='white',
                linewidth=0.3
            )
    
    # Add borough labels at approximate centers
    borough_centers = {
        'MANHATTAN': (40.78, -73.97),
        'BROOKLYN': (40.65, -73.95),
        'QUEENS': (40.73, -73.80),
        'BRONX': (40.85, -73.87),
        'STATEN IS.': (40.58, -74.15)
    }
    
    for borough, (lat, lon) in borough_centers.items():
        ax.annotate(
            borough, 
            (lon, lat),
            fontsize=12, 
            color='white', 
            fontweight='bold',
            ha='center',
            bbox=dict(boxstyle='round,pad=0.4', facecolor='#0D1117', 
                     edgecolor='white', alpha=0.9, linewidth=2)
        )
    
    ax.set_xlabel('Longitude', fontsize=12, color='white')
    ax.set_ylabel('Latitude', fontsize=12, color='white')
    ax.set_title('Borough Verification Map\n(Check if colors match geography)', 
                 fontsize=18, color='white', fontweight='bold', pad=20)
    ax.tick_params(colors='white')
    
    for spine in ax.spines.values():
        spine.set_color('#30363D')
    
    legend = ax.legend(loc='upper left', fontsize=10, frameon=True, 
                       facecolor='#161B22', edgecolor='#30363D',
                       title='Borough (Count)')
    legend.get_title().set_color('white')
    for text in legend.get_texts():
        text.set_color('white')
    
    ax.grid(color='#30363D', alpha=0.3)
    
    plt.tight_layout()
    plt.savefig('borough_verification.png', dpi=200, facecolor='#0D1117', bbox_inches='tight')
    plt.show()
    print("\n✅ Saved: borough_verification.png")
# Run verification
verify_borough_assignment(df_clean)

In [0]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots
import warnings
warnings.filterwarnings('ignore')

# =============================================================================
# CONSTANTS
# =============================================================================

BOROUGH_COLORS = {
    'Manhattan': '#E63946',
    'Brooklyn': '#457B9D',
    'Queens': '#2A9D8F',
    'Bronx': '#9B59B6',
    'Staten Island': '#F4A261',
    'Unknown': '#888888'
}

# Building Class Readable Names
BUILDING_CLASS_NAMES = {
    # ONE FAMILY DWELLINGS (A)
    'A0': 'One Family - Cape Cod',
    'A1': 'One Family - Two Story Detached',
    'A2': 'One Family - One Story',
    'A3': 'One Family - Large Suburban',
    'A4': 'One Family - City Residence',
    'A5': 'One Family - Attached/Semi-Detached',
    'A6': 'One Family - Summer Cottage',
    'A7': 'One Family - Mansion',
    'A8': 'One Family - Bungalow Colony',
    'A9': 'One Family - Miscellaneous',
    
    # TWO FAMILY DWELLINGS (B)
    'B1': 'Two Family - Brick',
    'B2': 'Two Family - Frame',
    'B3': 'Two Family - Converted',
    'B9': 'Two Family - Miscellaneous',
    
    # WALK-UP APARTMENTS (C)
    'C0': 'Walk-Up - Three Families',
    'C1': 'Walk-Up - Over 6 Units (No Stores)',
    'C2': 'Walk-Up - 5-6 Units',
    'C3': 'Walk-Up - 4 Units',
    'C4': 'Walk-Up - Old Law Tenement',
    'C5': 'Walk-Up - Converted Dwelling',
    'C6': 'Walk-Up - Cooperative',
    'C7': 'Walk-Up - With Commercial',
    'C8': 'Walk-Up - Co-op With Commercial',
    'C9': 'Walk-Up - Garden Apartment',
    
    # ELEVATOR APARTMENTS (D)
    'D0': 'Elevator - Co-op (Billing)',
    'D1': 'Elevator - Semi-Fireproof',
    'D2': 'Elevator - Artists In Residence',
    'D3': 'Elevator - Fireproof With Stores',
    'D4': 'Elevator - Cooperative',
    'D5': 'Elevator - Converted',
    'D6': 'Elevator - Fireproof',
    'D7': 'Elevator - With Commercial',
    'D8': 'Elevator - Co-op With Commercial',
    'D9': 'Elevator - Miscellaneous',
    
    # CONDOMINIUMS (R)
    'R0': 'Condo - Billing Lot',
    'R1': 'Condo - Residential Unit',
    'R2': 'Condo - Residential With Garage',
    'R3': 'Condo - Professional Unit',
    'R4': 'Condo - Commercial Unit',
    'R5': 'Condo - Misc Commercial',
    'R6': 'Condo - Garage/Parking',
    'R7': 'Condo - Rental',
    'R8': 'Condo - Other',
    'R9': 'Condo - Miscellaneous',
    
    # RETAIL/STORES (K)
    'K1': 'Retail - One Story',
    'K2': 'Retail - Two Story',
    'K3': 'Retail - Multi-Story',
    'K4': 'Retail - Predominant',
    'K5': 'Retail - Food Establishment',
    'K6': 'Retail - Shopping Center',
    'K7': 'Retail - Banking',
    'K8': 'Retail - Big Box',
    'K9': 'Retail - Miscellaneous',
    
    # MIXED USE STORES (S)
    'S0': 'Mixed Use - 1 Story Store',
    'S1': 'Mixed Use - 2 Story Store',
    'S2': 'Mixed Use - 3 Story Store',
    'S3': 'Mixed Use - 4 Story Store',
    'S4': 'Mixed Use - 5+ Story Store',
    'S5': 'Mixed Use - Store With Apartments',
    'S9': 'Mixed Use - Miscellaneous',
    
    # OFFICES (O)
    'O1': 'Office - 1 Story',
    'O2': 'Office - 2-6 Stories',
    'O3': 'Office - 7-19 Stories',
    'O4': 'Office - 20+ Stories (Skyscraper)',
    'O5': 'Office - With Apartments',
    'O6': 'Office - Building',
    'O7': 'Office - Professional Building',
    'O8': 'Office - Co-op',
    'O9': 'Office - Miscellaneous',
    
    # WAREHOUSES (E)
    'E1': 'Warehouse - Fireproof',
    'E2': 'Warehouse - Contractors',
    'E3': 'Warehouse - Semi-Fireproof',
    'E4': 'Warehouse - Metal Frame',
    'E7': 'Warehouse - Self Storage',
    'E9': 'Warehouse - Miscellaneous',
    
    # FACTORIES (F)
    'F1': 'Factory - Heavy Manufacturing',
    'F2': 'Factory - Special Construction',
    'F4': 'Factory - Semi-Fireproof',
    'F5': 'Factory - Light Manufacturing',
    'F8': 'Factory - Tank Farm',
    'F9': 'Factory - Miscellaneous',
    
    # GARAGES (G)
    'G0': 'Garage - Residential',
    'G1': 'Garage - Parking',
    'G2': 'Garage - Auto Repair',
    'G3': 'Garage - Gas Station',
    'G4': 'Garage - Parking Lot',
    'G5': 'Garage - Converted',
    'G6': 'Garage - Licensed Parking Lot',
    'G7': 'Garage - Truck Terminal',
    'G8': 'Garage - Car Sales/Rental',
    'G9': 'Garage - Miscellaneous',
    
    # HOTELS (H)
    'H1': 'Hotel - Luxury',
    'H2': 'Hotel - Full Service',
    'H3': 'Hotel - Limited Service',
    'H4': 'Hotel - Motel',
    'H5': 'Hotel - Private Club',
    'H6': 'Hotel - Apartment Hotel',
    'H7': 'Hotel - Apartment Hotel Co-op',
    'H8': 'Hotel - Dormitory',
    'H9': 'Hotel - Miscellaneous',
    
    # HEALTHCARE (I)
    'I1': 'Healthcare - Hospital',
    'I2': 'Healthcare - Infirmary',
    'I3': 'Healthcare - Dispensary',
    'I4': 'Healthcare - Staff Facility',
    'I5': 'Healthcare - Health Related',
    'I6': 'Healthcare - Nursing Home',
    'I7': 'Healthcare - Adult Care',
    'I9': 'Healthcare - Miscellaneous',
    
    # LOFTS (L)
    'L1': 'Loft - Over 8 Stories',
    'L2': 'Loft - Fireproof',
    'L3': 'Loft - Semi-Fireproof',
    'L8': 'Loft - With Retail',
    'L9': 'Loft - Miscellaneous',
    
    # RELIGIOUS (M)
    'M1': 'Religious - Church',
    'M2': 'Religious - Mission',
    'M3': 'Religious - Parsonage',
    'M4': 'Religious - Convent',
    'M9': 'Religious - Miscellaneous',
    
    # INSTITUTIONAL (N)
    'N1': 'Institutional - Asylum',
    'N2': 'Institutional - Homes for Indigent',
    'N3': 'Institutional - Orphanage',
    'N4': 'Institutional - Detention',
    'N9': 'Institutional - Miscellaneous',
    
    # RECREATION (P)
    'P1': 'Recreation - Stadium/Arena',
    'P2': 'Recreation - Beach Property',
    'P3': 'Recreation - Outdoor Pool',
    'P4': 'Recreation - Beach Club',
    'P5': 'Recreation - Golf Course',
    'P6': 'Recreation - Amusement Park',
    'P7': 'Recreation - Cemetery',
    'P8': 'Recreation - Marina',
    'P9': 'Recreation - Miscellaneous',
    
    # PARKS (Q)
    'Q0': 'Parks - Open Space',
    'Q1': 'Parks - Parks/Recreation',
    'Q2': 'Parks - Playground',
    'Q3': 'Parks - Outdoor Pool',
    'Q4': 'Parks - Beach',
    'Q5': 'Parks - Golf Course',
    'Q6': 'Parks - Developed Waterfront',
    'Q7': 'Parks - Undeveloped Property',
    'Q8': 'Parks - Tennis Court',
    'Q9': 'Parks - Miscellaneous',
    
    # TRANSPORTATION (T)
    'T1': 'Transportation - Airport',
    'T2': 'Transportation - Piers/Docks',
    'T9': 'Transportation - Miscellaneous',
    
    # UTILITY (U)
    'U0': 'Utility - Company Property',
    'U1': 'Utility - Gas/Electric',
    'U2': 'Utility - Telecommunications',
    'U3': 'Utility - Water/Sewage',
    'U4': 'Utility - Communications',
    'U5': 'Utility - Other',
    'U6': 'Utility - Public Facility',
    'U7': 'Utility - Land Under Water',
    'U8': 'Utility - New York Utility',
    'U9': 'Utility - Miscellaneous',
    
    # VACANT (V)
    'V0': 'Vacant - Zoned Residential',
    'V1': 'Vacant - Zoned Commercial',
    'V2': 'Vacant - Zoned Mixed',
    'V3': 'Vacant - Primarily Residential',
    'V4': 'Vacant - Police/Fire',
    'V5': 'Vacant - School',
    'V6': 'Vacant - Library/Hospital',
    'V7': 'Vacant - Cultural',
    'V8': 'Vacant - Government',
    'V9': 'Vacant - Miscellaneous',
    
    # EDUCATION (W)
    'W1': 'School - Public',
    'W2': 'School - Parochial',
    'W3': 'School - Private Academy',
    'W4': 'School - Training',
    'W5': 'School - City University',
    'W6': 'School - College',
    'W7': 'School - Theological Seminary',
    'W8': 'School - Other Private',
    'W9': 'School - Miscellaneous',
    
    # OUTDOOR RECREATION (X)
    'X1': 'Outdoor - Pool',
    'X2': 'Outdoor - Tennis',
    'X3': 'Outdoor - Handball',
    'X8': 'Outdoor - Track',
    'X9': 'Outdoor - Miscellaneous',
    
    # GOVERNMENT (Y)
    'Y1': 'Government - Fire Department',
    'Y2': 'Government - Police Department',
    'Y3': 'Government - Prison/Jail',
    'Y4': 'Government - Military',
    'Y5': 'Government - Real Estate Dept',
    'Y6': 'Government - Sanitation',
    'Y7': 'Government - Parks Dept',
    'Y8': 'Government - Ports/Transportation',
    'Y9': 'Government - Miscellaneous',
    
    # MIXED USE (Z)
    'Z0': 'Mixed - Residential/Commercial',
    'Z1': 'Mixed - Warehouse/Residential',
    'Z2': 'Mixed - Commercial/Manufacturing',
    'Z3': 'Mixed - Residential/Industrial',
    'Z4': 'Mixed - Commercial/Industrial',
    'Z5': 'Mixed - Residential/Commercial/Industrial',
    'Z7': 'Mixed - Residential/Parking',
    'Z8': 'Mixed - Commercial/Parking',
    'Z9': 'Mixed - Miscellaneous',
    
    # THEATERS (J)
    'J1': 'Theatre - Art Type',
    'J2': 'Theatre - Art Type Fireproof',
    'J3': 'Theatre - Art Type Semi-Fireproof',
    'J4': 'Theatre - Legitimate',
    'J5': 'Theatre - Movie Fireproof',
    'J6': 'Theatre - Movie Open',
    'J7': 'Theatre - Drive-In',
    'J8': 'Theatre - Loft',
    'J9': 'Theatre - Miscellaneous',
}

# Simplified category based on first letter
BUILDING_CATEGORY = {
    'A': 'Single Family Home',
    'B': 'Two Family Home',
    'C': 'Walk-Up Apartment',
    'D': 'Elevator Apartment',
    'R': 'Condo',
    'S': 'Mixed Use (Residential/Retail)',
    'K': 'Retail',
    'O': 'Office',
    'E': 'Warehouse',
    'F': 'Factory/Industrial',
    'G': 'Garage/Parking',
    'H': 'Hotel',
    'L': 'Loft',
    'M': 'Religious',
    'V': 'Vacant Land',
    'W': 'School/Education',
    'I': 'Healthcare',
    'J': 'Theater',
    'N': 'Institutional',
    'P': 'Recreation (Outdoor)',
    'Q': 'Parks',
    'T': 'Transportation',
    'U': 'Utility',
    'Y': 'Government',
    'Z': 'Mixed Use (Other)',
    'X': 'Outdoor Recreation'
}

# Dark theme settings
plt.rcParams.update({
    'font.family': 'sans-serif',
    'font.size': 12,
    'axes.titlesize': 16,
    'axes.titleweight': 'bold',
    'axes.spines.top': False,
    'axes.spines.right': False,
    'figure.facecolor': '#0D1117',
    'axes.facecolor': '#0D1117',
    'axes.edgecolor': '#30363D',
    'axes.labelcolor': 'white',
    'text.color': 'white',
    'xtick.color': 'white',
    'ytick.color': 'white'
})

# =============================================================================
# LOAD CLEANED DATA
# =============================================================================

df = pd.read_csv('nyc_housing_minimal_clean.csv')

print("="*80)
print("📂 CLEANED DATA LOADED")
print("="*80)
print(f"Total records: {len(df):,}")
print(f"Columns: {list(df.columns)}\n")

# =============================================================================
# CLASSIFY PROPERTY TYPES
# =============================================================================

def classify_property_types(df):
    """Classify NYC properties using building class codes."""
    
    print("="*80)
    print("🏠 PROPERTY TYPE CLASSIFICATION")
    print("="*80)
    
    df_typed = df.copy()
    
    # Show building classes
    print("\n📊 Top 20 Building Classes:")
    print("-" * 70)
    print(f"{'Code':<6} {'Property Type':<45} {'Count':>8} {'%':>6}")
    print("-" * 70)
    
    class_counts = df_typed['bldgclass'].value_counts().head(20)
    
    for bldg_class, count in class_counts.items():
        name = BUILDING_CLASS_NAMES.get(bldg_class, 'Unknown Type')
        pct = count / len(df_typed) * 100
        print(f"{bldg_class:<6} {name:<45} {count:>8,} {pct:>5.1f}%")
    
    # Add detailed property type
    df_typed['property_type'] = df_typed['bldgclass'].map(BUILDING_CLASS_NAMES)
    df_typed['property_type'] = df_typed['property_type'].fillna('Other/Unknown')
    
    # Add simplified category
    def get_category(bldg_class):
        if pd.isna(bldg_class) or len(str(bldg_class)) == 0:
            return 'Unknown'
        first_letter = str(bldg_class)[0].upper()
        return BUILDING_CATEGORY.get(first_letter, 'Other')
    
    df_typed['property_category'] = df_typed['bldgclass'].apply(get_category)
    
    # Add use type
    def get_use_type(bldg_class):
        if pd.isna(bldg_class) or len(str(bldg_class)) == 0:
            return 'Unknown'
        
        first_letter = str(bldg_class)[0].upper()
        
        if first_letter in ['A', 'B', 'C', 'D', 'R']:
            return 'Residential'
        elif first_letter in ['O', 'K', 'E', 'F', 'G', 'H', 'I', 'J', 'L']:
            return 'Commercial'
        elif first_letter in ['S', 'Z']:
            return 'Mixed Use'
        else:
            return 'Special Purpose'
    
    df_typed['use_type'] = df_typed['bldgclass'].apply(get_use_type)
    
    # ==========================================================================
    # SUMMARIES
    # ==========================================================================
    
    print("\n" + "="*80)
    print("📊 PROPERTY CATEGORY SUMMARY")
    print("="*80)
    print(f"\n{'Category':<40} {'Count':>10} {'%':>7} {'Median Price':>15}")
    print("-" * 75)
    
    for ptype in df_typed['property_category'].value_counts().head(12).index:
        subset = df_typed[df_typed['property_category'] == ptype]
        count = len(subset)
        pct = count / len(df_typed) * 100
        median_price = subset['sale_price'].median()
        print(f"{ptype:<40} {count:>10,} {pct:>6.1f}% ${median_price:>13,.0f}")
    
    print("\n" + "="*80)
    print("🏘️ USE TYPE SUMMARY")
    print("="*80)
    print(f"\n{'Use Type':<25} {'Count':>10} {'%':>7} {'Median Price':>15}")
    print("-" * 60)
    
    for use in df_typed['use_type'].value_counts().index:
        subset = df_typed[df_typed['use_type'] == use]
        count = len(subset)
        pct = count / len(df_typed) * 100
        median_price = subset['sale_price'].median()
        print(f"{use:<25} {count:>10,} {pct:>6.1f}% ${median_price:>13,.0f}")
    
    print("\n" + "="*80)
    print("🗺️ TOP PROPERTY TYPES BY BOROUGH")
    print("="*80)
    
    for borough in ['Manhattan', 'Brooklyn', 'Queens', 'Bronx', 'Staten Island']:
        borough_data = df_typed[df_typed['borough_name'] == borough]
        
        if len(borough_data) > 0:
            print(f"\n{borough} ({len(borough_data):,} properties):")
            top_types = borough_data['property_category'].value_counts().head(5)
            for ptype, count in top_types.items():
                pct = count / len(borough_data) * 100
                print(f"  • {ptype:<35} {count:>6,} ({pct:>5.1f}%)")
    
    return df_typed


# Apply classification
df = classify_property_types(df)

# =============================================================================
# QUICK DATA SUMMARY
# =============================================================================

print("\n" + "="*80)
print("📈 DATA SUMMARY")
print("="*80)

print(f"\nTotal Records: {len(df):,}")
print(f"\nPrice Statistics:")
print(f"  Median: ${df['sale_price'].median():,.0f}")
print(f"  Mean:   ${df['sale_price'].mean():,.0f}")
print(f"  Min:    ${df['sale_price'].min():,.0f}")
print(f"  Max:    ${df['sale_price'].max():,.0f}")

if 'price_per_sqft' in df.columns:
    valid_pps = df[df['price_per_sqft'] > 0]['price_per_sqft']
    print(f"\nPrice/SqFt (excluding $0):")
    print(f"  Median: ${valid_pps.median():,.0f}")
    print(f"  Mean:   ${valid_pps.mean():,.0f}")
    print(f"  Min:    ${valid_pps.min():,.0f}")
    print(f"  Max:    ${valid_pps.max():,.0f}")

print(f"\nBorough Distribution:")
for borough, count in df['borough_name'].value_counts().items():
    pct = count / len(df) * 100
    print(f"  {borough:<20} {count:>8,} ({pct:>5.1f}%)")

# Save enhanced dataset
df.to_csv('nyc_housing_classified.csv', index=False)

print("\n" + "="*80)
print("✅ COMPLETE")
print("="*80)
print(f"\n💾 Saved: nyc_housing_classified.csv")
print(f"📊 Records: {len(df):,}")
print(f"\nNew columns added:")
print("  • property_type (detailed, e.g., 'Elevator - Cooperative')")
print("  • property_category (simplified, e.g., 'Elevator Apartment')")
print("  • use_type (Residential/Commercial/Mixed Use/Special Purpose)")
print("\n🎉 Ready for analysis!")

In [0]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots
import warnings
warnings.filterwarnings('ignore')

# =============================================================================
# LOAD DATA (from Kaggle - already cleaned)
# =============================================================================

df = pd.read_csv('/Volumes/workspace/default/nyc-housing/nyc_housing_base.csv')

print("="*80)
print("📂 KAGGLE DATASET LOADED (Pre-Cleaned)")
print("="*80)
print(f"Total records: {len(df):,}")
print(f"Source: NYC Housing Prices (Kaggle)")
print(f"Columns: {list(df.columns)}\n")

# =============================================================================
# CONSTANTS & MAPPINGS
# =============================================================================

BOROUGH_NAMES = {
    1: 'Manhattan',
    2: 'Bronx',
    3: 'Brooklyn',
    4: 'Queens',
    5: 'Staten Island'
}

BOROUGH_COLORS = {
    'Manhattan': '#E63946',
    'Brooklyn': '#457B9D',
    'Queens': '#2A9D8F',
    'Bronx': '#9B59B6',
    'Staten Island': '#F4A261'
}

# Building Class Names - Full NYC Classification
BUILDING_CLASS_NAMES = {
    # ONE FAMILY DWELLINGS (A)
    'A0': 'One Family - Cape Cod',
    'A1': 'One Family - Two Story Detached',
    'A2': 'One Family - One Story',
    'A3': 'One Family - Large Suburban',
    'A4': 'One Family - City Residence',
    'A5': 'One Family - Attached/Semi-Detached',
    'A6': 'One Family - Summer Cottage',
    'A7': 'One Family - Mansion',
    'A8': 'One Family - Bungalow Colony',
    'A9': 'One Family - Miscellaneous',
    
    # TWO FAMILY DWELLINGS (B)
    'B1': 'Two Family - Brick',
    'B2': 'Two Family - Frame',
    'B3': 'Two Family - Converted',
    'B9': 'Two Family - Miscellaneous',
    
    # WALK-UP APARTMENTS (C)
    'C0': 'Walk-Up - Three Families',
    'C1': 'Walk-Up - Over 6 Units',
    'C2': 'Walk-Up - 5-6 Units',
    'C3': 'Walk-Up - 4 Units',
    'C4': 'Walk-Up - Old Law Tenement',
    'C5': 'Walk-Up - Converted',
    'C6': 'Walk-Up - Cooperative',
    'C7': 'Walk-Up - With Commercial',
    'C8': 'Walk-Up - Co-op With Commercial',
    'C9': 'Walk-Up - Garden Apartment',
    
    # ELEVATOR APARTMENTS (D)
    'D0': 'Elevator - Co-op',
    'D1': 'Elevator - Semi-Fireproof',
    'D2': 'Elevator - Artists In Residence',
    'D3': 'Elevator - Fireproof With Stores',
    'D4': 'Elevator - Cooperative',
    'D5': 'Elevator - Converted',
    'D6': 'Elevator - Fireproof',
    'D7': 'Elevator - With Commercial',
    'D8': 'Elevator - Co-op With Commercial',
    'D9': 'Elevator - Miscellaneous',
    
    # CONDOS (R)
    'R1': 'Condo - Residential',
    'R2': 'Condo - With Garage',
    'R3': 'Condo - Professional Unit',
    'R4': 'Condo - Commercial',
    'R6': 'Condo - Garage/Parking',
    'R9': 'Condo - Miscellaneous',
    
    # MIXED USE (S)
    'S0': 'Mixed Use - 1 Story Store',
    'S1': 'Mixed Use - 2 Story Store',
    'S2': 'Mixed Use - 3 Story Store',
    'S3': 'Mixed Use - 4 Story Store',
    'S4': 'Mixed Use - 5+ Story Store',
    'S5': 'Mixed Use - Store With Apartments',
    'S9': 'Mixed Use - Miscellaneous',
    
    # RETAIL (K)
    'K1': 'Retail - One Story',
    'K2': 'Retail - Two Story',
    'K3': 'Retail - Multi-Story',
    'K4': 'Retail - Predominant',
    'K5': 'Retail - Food Establishment',
    'K6': 'Retail - Shopping Center',
    'K9': 'Retail - Miscellaneous',
    
    # OFFICE (O)
    'O1': 'Office - 1 Story',
    'O2': 'Office - 2-6 Stories',
    'O3': 'Office - 7-19 Stories',
    'O4': 'Office - 20+ Stories',
    'O5': 'Office - With Apartments',
    'O9': 'Office - Miscellaneous',
    
    # WAREHOUSE (E)
    'E1': 'Warehouse - Fireproof',
    'E4': 'Warehouse - Metal Frame',
    'E7': 'Warehouse - Self Storage',
    'E9': 'Warehouse - Miscellaneous',
    
    # FACTORY (F)
    'F1': 'Factory - Heavy Manufacturing',
    'F5': 'Factory - Light Manufacturing',
    'F9': 'Factory - Miscellaneous',
    
    # GARAGE (G)
    'G0': 'Garage - Residential',
    'G1': 'Garage - Parking',
    'G2': 'Garage - Auto Repair',
    'G9': 'Garage - Miscellaneous',
    
    # HOTEL (H)
    'H1': 'Hotel - Luxury',
    'H2': 'Hotel - Full Service',
    'H3': 'Hotel - Limited Service',
    'H4': 'Hotel - Motel',
    'H9': 'Hotel - Miscellaneous',
    
    # LOFT (L)
    'L1': 'Loft - Over 8 Stories',
    'L2': 'Loft - Fireproof',
    'L3': 'Loft - Semi-Fireproof',
    'L8': 'Loft - With Retail',
    'L9': 'Loft - Miscellaneous',
}

# Simplified categories
BUILDING_CATEGORY = {
    'A': 'Single Family Home',
    'B': 'Two Family Home',
    'C': 'Walk-Up Apartment',
    'D': 'Elevator Apartment',
    'R': 'Condo',
    'S': 'Mixed Use',
    'K': 'Retail',
    'O': 'Office',
    'E': 'Warehouse',
    'F': 'Factory/Industrial',
    'G': 'Garage/Parking',
    'H': 'Hotel',
    'L': 'Loft',
    'M': 'Religious',
    'V': 'Vacant Land',
    'W': 'School/Education',
    'I': 'Healthcare',
    'J': 'Theater',
    'N': 'Institutional',
    'P': 'Recreation',
    'Q': 'Parks',
    'T': 'Transportation',
    'U': 'Utility',
    'Y': 'Government',
    'Z': 'Mixed Use (Other)',
}

# =============================================================================
# PROCESSING
# =============================================================================

def assign_borough_from_coordinates(lat, lon):
    """Assign borough based on coordinates."""
    if pd.isna(lat) or pd.isna(lon):
        return 'Unknown'
    if lon < -74.05:
        return 'Staten Island'
    if lat >= 40.79 and lon > -74.05:
        return 'Bronx'
    if 40.70 <= lat <= 40.88 and -74.02 <= lon <= -73.93:
        return 'Manhattan'
    if lat < 40.70 and lon > -74.05 and lon < -73.85:
        return 'Brooklyn'
    if lat < 40.74 and lon <= -73.92:
        return 'Brooklyn'
    if lon >= -73.88:
        return 'Queens'
    if lat < 40.74:
        return 'Brooklyn'
    return 'Queens'

print("🔧 Processing data...")

# Convert numeric columns
numeric_cols = ['sale_price', 'bldgarea', 'lotarea', 'resarea', 'comarea',
                'yearbuilt', 'building_age', 'numfloors', 'unitsres', 
                'unitstotal', 'latitude', 'longitude']

for col in numeric_cols:
    if col in df.columns:
        df[col] = pd.to_numeric(df[col], errors='coerce')

# Remove invalid building areas
initial_count = len(df)
df = df[~((df['bldgarea'].notna()) & (df['bldgarea'] <= 0))].copy()
print(f"✓ Removed {initial_count - len(df)} records with invalid building area")

# Assign boroughs
has_coords = df['latitude'].notna() & df['longitude'].notna()
df.loc[has_coords, 'borough_name'] = df[has_coords].apply(
    lambda row: assign_borough_from_coordinates(row['latitude'], row['longitude']), axis=1
)

if 'borough_x' in df.columns:
    no_coords = ~has_coords & df['borough_name'].isna()
    df.loc[no_coords, 'borough_name'] = df.loc[no_coords, 'borough_x'].map(BOROUGH_NAMES)

df['borough_name'] = df['borough_name'].fillna('Unknown')

# =============================================================================
# BUILDING CLASSIFICATION
# =============================================================================

print("🏠 Classifying buildings...")

# Add detailed building type
df['building_type'] = df['bldgclass'].map(BUILDING_CLASS_NAMES)
df['building_type'] = df['building_type'].fillna('Other')

# Add simplified category
def get_building_category(bldg_class):
    if pd.isna(bldg_class) or len(str(bldg_class)) == 0:
        return 'Unknown'
    first_letter = str(bldg_class)[0].upper()
    return BUILDING_CATEGORY.get(first_letter, 'Other')

df['building_category'] = df['bldgclass'].apply(get_building_category)

# Add use type
def get_use_type(bldg_class):
    if pd.isna(bldg_class) or len(str(bldg_class)) == 0:
        return 'Unknown'
    first_letter = str(bldg_class)[0].upper()
    
    if first_letter in ['A', 'B', 'C', 'D', 'R']:
        return 'Residential'
    elif first_letter in ['O', 'K', 'E', 'F', 'G', 'H', 'I', 'J', 'L']:
        return 'Commercial'
    elif first_letter in ['S', 'Z']:
        return 'Mixed Use'
    else:
        return 'Special Purpose'

df['use_type'] = df['bldgclass'].apply(get_use_type)

# =============================================================================
# CALCULATE METRICS
# =============================================================================

# Price per square foot
has_area = df['bldgarea'] > 0
df.loc[has_area, 'price_per_sqft'] = df.loc[has_area, 'sale_price'] / df.loc[has_area, 'bldgarea']

# Property segment
def classify_property(row):
    area = row['bldgarea']
    pps = row['price_per_sqft']
    
    if pd.notna(area) and area > 500000:
        return 'Large Commercial/Portfolio'
    if pd.notna(pps) and pps < 10:
        return 'Air Rights/Land/Special'
    if pd.notna(pps) and pps > 2000:
        return 'Luxury'
    if pd.notna(pps) and 100 <= pps <= 2000:
        return 'Standard'
    if pd.notna(pps) and 10 <= pps < 100:
        return 'Budget'
    return 'Other'

df['price_segment'] = df.apply(classify_property, axis=1)

# Price tiers
df['price_tier'] = pd.cut(
    df['sale_price'],
    bins=[0, 300000, 600000, 900000, 1500000, 3000000, float('inf')],
    labels=['Under $300K', '$300K-600K', '$600K-900K', '$900K-1.5M', '$1.5M-3M', 'Over $3M']
)

# Size categories
df['size_category'] = pd.cut(
    df['bldgarea'],
    bins=[0, 800, 1500, 2500, 5000, 10000, float('inf')],
    labels=['Small (<800)', 'Medium (800-1.5K)', 'Large (1.5-2.5K)', 
            'Very Large (2.5-5K)', 'Massive (5-10K)', 'Commercial (10K+)']
)

# Age categories
df['age_category'] = pd.cut(
    df['building_age'],
    bins=[0, 10, 25, 50, 75, 100, float('inf')],
    labels=['New (0-10)', 'Recent (10-25)', 'Modern (25-50)', 
            'Mature (50-75)', 'Old (75-100)', 'Historic (100+)']
)

print("✓ All classifications complete!")

# =============================================================================
# DATA SUMMARY
# =============================================================================

print("\n" + "="*80)
print("📊 DATA SUMMARY")
print("="*80)

print(f"\nTotal Properties: {len(df):,}")

print(f"\n💰 Price Statistics:")
print(f"  Range:  ${df['sale_price'].min():,.0f} - ${df['sale_price'].max():,.0f}")
print(f"  Median: ${df['sale_price'].median():,.0f}")
print(f"  Mean:   ${df['sale_price'].mean():,.0f}")

print(f"\n🏘️ Borough Distribution:")
for borough, count in df['borough_name'].value_counts().items():
    pct = count / len(df) * 100
    median_price = df[df['borough_name'] == borough]['sale_price'].median()
    print(f"  {borough:<20} {count:>8,} ({pct:>5.1f}%)  Median: ${median_price:>10,.0f}")

print(f"\n🏠 Building Category Distribution:")
for cat, count in df['building_category'].value_counts().head(10).items():
    pct = count / len(df) * 100
    median_price = df[df['building_category'] == cat]['sale_price'].median()
    print(f"  {cat:<25} {count:>8,} ({pct:>5.1f}%)  Median: ${median_price:>10,.0f}")

print(f"\n🏢 Use Type Distribution:")
for use, count in df['use_type'].value_counts().items():
    pct = count / len(df) * 100
    median_price = df[df['use_type'] == use]['sale_price'].median()
    print(f"  {use:<20} {count:>8,} ({pct:>5.1f}%)  Median: ${median_price:>10,.0f}")

# =============================================================================
# 1. INTERACTIVE MAP OF ALL PROPERTIES
# =============================================================================

print("\n" + "="*80)
print("🗺️ CREATING INTERACTIVE MAP...")
print("="*80)

# Sample for performance (too many points slows down the map)
df_map = df[df['latitude'].notna() & df['longitude'].notna()].copy()
if len(df_map) > 10000:
    df_map = df_map.sample(10000, random_state=42)
    print(f"  Sampled {len(df_map):,} properties for map performance")

# Create hover text
df_map['hover_text'] = (
    '<b>' + df_map['building_category'].astype(str) + '</b><br>' +
    'Price: $' + df_map['sale_price'].apply(lambda x: f'{x:,.0f}') + '<br>' +
    'Size: ' + df_map['bldgarea'].apply(lambda x: f'{x:,.0f}' if pd.notna(x) else 'N/A') + ' sqft<br>' +
    'Borough: ' + df_map['borough_name'].astype(str) + '<br>' +
    'Type: ' + df_map['bldgclass'].astype(str)
)

# Create the map
fig_map = px.scatter_mapbox(
    df_map,
    lat='latitude',
    lon='longitude',
    color='borough_name',
    color_discrete_map=BOROUGH_COLORS,
    size='sale_price',
    size_max=15,
    hover_name='building_category',
    hover_data={
        'sale_price': ':$,.0f',
        'bldgarea': ':,.0f',
        'price_per_sqft': ':$,.0f',
        'building_type': True,
        'latitude': False,
        'longitude': False
    },
    zoom=10,
    center={'lat': 40.7128, 'lon': -73.9860},
    title='NYC Property Sales Map',
    mapbox_style='carto-darkmatter'
)

fig_map.update_layout(
    height=700,
    margin=dict(l=0, r=0, t=50, b=0),
    legend=dict(
        yanchor="top",
        y=0.99,
        xanchor="left",
        x=0.01,
        bgcolor="rgba(0,0,0,0.5)"
    ),
    title_font_size=20
)

fig_map.write_html('nyc_property_map.html')
print("  ✓ Saved: nyc_property_map.html")

# =============================================================================
# 2. MAP BY BUILDING CATEGORY
# =============================================================================

fig_map_category = px.scatter_mapbox(
    df_map,
    lat='latitude',
    lon='longitude',
    color='building_category',
    hover_name='building_type',
    hover_data={
        'sale_price': ':$,.0f',
        'bldgarea': ':,.0f',
        'borough_name': True,
        'latitude': False,
        'longitude': False
    },
    zoom=10,
    center={'lat': 40.7128, 'lon': -73.9860},
    title='NYC Properties by Building Type',
    mapbox_style='carto-darkmatter'
)

fig_map_category.update_layout(
    height=700,
    margin=dict(l=0, r=0, t=50, b=0),
    legend=dict(
        yanchor="top",
        y=0.99,
        xanchor="left",
        x=0.01,
        bgcolor="rgba(0,0,0,0.5)"
    )
)

fig_map_category.write_html('nyc_property_map_by_type.html')
print("  ✓ Saved: nyc_property_map_by_type.html")

# =============================================================================
# 3. PRICE HEATMAP
# =============================================================================

fig_heatmap = px.density_mapbox(
    df_map,
    lat='latitude',
    lon='longitude',
    z='price_per_sqft',
    radius=10,
    center={'lat': 40.7128, 'lon': -73.9860},
    zoom=10,
    mapbox_style='carto-darkmatter',
    title='NYC Price per SqFt Heatmap',
    color_continuous_scale='Inferno'
)

fig_heatmap.update_layout(
    height=700,
    margin=dict(l=0, r=0, t=50, b=0)
)

fig_heatmap.write_html('nyc_price_heatmap.html')
print("  ✓ Saved: nyc_price_heatmap.html")

# =============================================================================
# 4. ANALYTICS DASHBOARD
# =============================================================================

print("\n" + "="*80)
print("📈 CREATING ANALYTICS DASHBOARD...")
print("="*80)

# Create subplots
fig_dashboard = make_subplots(
    rows=3, cols=2,
    subplot_titles=(
        'Price Distribution by Borough',
        'Building Types Distribution',
        'Price per SqFt by Borough',
        'Property Age Distribution',
        'Price vs Size (log scale)',
        'Sales by Price Tier'
    ),
    specs=[
        [{"type": "box"}, {"type": "pie"}],
        [{"type": "box"}, {"type": "bar"}],
        [{"type": "scatter"}, {"type": "bar"}]
    ],
    vertical_spacing=0.1,
    horizontal_spacing=0.1
)

# 1. Price Distribution by Borough (Box Plot)
for borough in ['Manhattan', 'Brooklyn', 'Queens', 'Bronx', 'Staten Island']:
    data = df[df['borough_name'] == borough]['sale_price']
    if len(data) > 0:
        fig_dashboard.add_trace(
            go.Box(
                y=data,
                name=borough,
                marker_color=BOROUGH_COLORS[borough],
                showlegend=False
            ),
            row=1, col=1
        )

# 2. Building Types Distribution (Pie Chart)
building_counts = df['building_category'].value_counts().head(8)
fig_dashboard.add_trace(
    go.Pie(
        labels=building_counts.index,
        values=building_counts.values,
        hole=0.4,
        textinfo='percent+label',
        showlegend=False
    ),
    row=1, col=2
)

# 3. Price per SqFt by Borough (Box Plot)
df_valid_pps = df[df['price_per_sqft'].notna() & (df['price_per_sqft'] > 0) & (df['price_per_sqft'] < 5000)]
for borough in ['Manhattan', 'Brooklyn', 'Queens', 'Bronx', 'Staten Island']:
    data = df_valid_pps[df_valid_pps['borough_name'] == borough]['price_per_sqft']
    if len(data) > 0:
        fig_dashboard.add_trace(
            go.Box(
                y=data,
                name=borough,
                marker_color=BOROUGH_COLORS[borough],
                showlegend=False
            ),
            row=2, col=1
        )

# 4. Property Age Distribution (Bar Chart)
age_counts = df['age_category'].value_counts().sort_index()
fig_dashboard.add_trace(
    go.Bar(
        x=age_counts.index.astype(str),
        y=age_counts.values,
        marker_color='#2A9D8F',
        showlegend=False
    ),
    row=2, col=2
)

# 5. Price vs Size Scatter
df_scatter = df[(df['bldgarea'] > 0) & (df['bldgarea'] < 50000) & (df['sale_price'] < 10000000)].sample(min(2000, len(df)))
fig_dashboard.add_trace(
    go.Scatter(
        x=df_scatter['bldgarea'],
        y=df_scatter['sale_price'],
        mode='markers',
        marker=dict(
            size=5,
            color=df_scatter['borough_name'].map(BOROUGH_COLORS),
            opacity=0.5
        ),
        showlegend=False
    ),
    row=3, col=1
)

# 6. Sales by Price Tier (Bar Chart)
tier_counts = df['price_tier'].value_counts().sort_index()
fig_dashboard.add_trace(
    go.Bar(
        x=tier_counts.index.astype(str),
        y=tier_counts.values,
        marker_color='#E63946',
        showlegend=False
    ),
    row=3, col=2
)

# Update layout
fig_dashboard.update_layout(
    height=1200,
    title_text='NYC Housing Market Analytics Dashboard',
    title_font_size=24,
    template='plotly_dark',
    showlegend=False
)

# Update axes
fig_dashboard.update_yaxes(title_text='Sale Price ($)', row=1, col=1)
fig_dashboard.update_yaxes(title_text='Price per SqFt ($)', row=2, col=1)
fig_dashboard.update_yaxes(title_text='Count', row=2, col=2)
fig_dashboard.update_xaxes(title_text='Building Area (sqft)', row=3, col=1)
fig_dashboard.update_yaxes(title_text='Sale Price ($)', row=3, col=1)
fig_dashboard.update_yaxes(title_text='Count', row=3, col=2)

fig_dashboard.write_html('nyc_analytics_dashboard.html')
print("  ✓ Saved: nyc_analytics_dashboard.html")

# =============================================================================
# 5. BUILDING TYPE ANALYSIS
# =============================================================================

print("\n" + "="*80)
print("🏠 CREATING BUILDING TYPE ANALYSIS...")
print("="*80)

# Building type stats
building_stats = df.groupby('building_category').agg({
    'sale_price': ['count', 'median', 'mean'],
    'price_per_sqft': 'median',
    'bldgarea': 'median',
    'building_age': 'median'
}).round(0)

building_stats.columns = ['Count', 'Median Price', 'Mean Price', 'Median $/SqFt', 'Median SqFt', 'Median Age']
building_stats = building_stats.sort_values('Count', ascending=False)

print("\n📊 Building Category Statistics:")
print("-" * 100)
print(f"{'Category':<25} {'Count':>10} {'Median Price':>15} {'$/SqFt':>12} {'Median SqFt':>12} {'Median Age':>12}")
print("-" * 100)
for cat, row in building_stats.head(12).iterrows():
    print(f"{cat:<25} {int(row['Count']):>10,} ${row['Median Price']:>13,.0f} ${row['Median $/SqFt']:>10,.0f} {row['Median SqFt']:>11,.0f} {row['Median Age']:>11.0f}")

# Building category by borough heatmap
fig_category_borough = px.histogram(
    df,
    x='borough_name',
    color='building_category',
    barmode='stack',
    title='Building Types by Borough',
    color_discrete_sequence=px.colors.qualitative.Set2
)

fig_category_borough.update_layout(
    height=600,
    template='plotly_dark',
    xaxis_title='Borough',
    yaxis_title='Number of Properties',
    legend_title='Building Category'
)

fig_category_borough.write_html('nyc_building_types_by_borough.html')
print("  ✓ Saved: nyc_building_types_by_borough.html")

# =============================================================================
# 6. PRICE ANALYSIS BY BUILDING TYPE
# =============================================================================

# Top building types price comparison
top_categories = df['building_category'].value_counts().head(8).index.tolist()
df_top = df[df['building_category'].isin(top_categories)]

fig_price_type = px.box(
    df_top,
    x='building_category',
    y='sale_price',
    color='building_category',
    title='Price Distribution by Building Type (Top 8)',
    color_discrete_sequence=px.colors.qualitative.Set1
)

fig_price_type.update_layout(
    height=600,
    template='plotly_dark',
    showlegend=False,
    xaxis_title='Building Category',
    yaxis_title='Sale Price ($)',
    yaxis=dict(range=[0, 5000000])  # Cap for visibility
)

fig_price_type.write_html('nyc_price_by_building_type.html')
print("  ✓ Saved: nyc_price_by_building_type.html")

# =============================================================================
# 7. BOROUGH DEEP DIVE
# =============================================================================

print("\n" + "="*80)
print("🗽 CREATING BOROUGH ANALYSIS...")
print("="*80)

fig_borough = make_subplots(
    rows=2, cols=3,
    subplot_titles=['Manhattan', 'Brooklyn', 'Queens', 'Bronx', 'Staten Island', 'Borough Comparison'],
    specs=[
        [{"type": "pie"}, {"type": "pie"}, {"type": "pie"}],
        [{"type": "pie"}, {"type": "pie"}, {"type": "bar"}]
    ]
)

positions = [(1,1), (1,2), (1,3), (2,1), (2,2)]
boroughs = ['Manhattan', 'Brooklyn', 'Queens', 'Bronx', 'Staten Island']

for i, borough in enumerate(boroughs):
    borough_data = df[df['borough_name'] == borough]
    cat_counts = borough_data['building_category'].value_counts().head(5)
    
    row, col = positions[i]
    fig_borough.add_trace(
        go.Pie(
            labels=cat_counts.index,
            values=cat_counts.values,
            name=borough,
            hole=0.3,
            textinfo='percent',
            showlegend=False
        ),
        row=row, col=col
    )

# Borough comparison bar chart
borough_medians = df.groupby('borough_name')['sale_price'].median().sort_values(ascending=True)
fig_borough.add_trace(
    go.Bar(
        x=borough_medians.values,
        y=borough_medians.index,
        orientation='h',
        marker_color=[BOROUGH_COLORS.get(b, '#888') for b in borough_medians.index],
        showlegend=False
    ),
    row=2, col=3
)

fig_borough.update_layout(
    height=800,
    title_text='Building Types by Borough',
    title_font_size=20,
    template='plotly_dark'
)

fig_borough.write_html('nyc_borough_analysis.html')
print("  ✓ Saved: nyc_borough_analysis.html")

# =============================================================================
# 8. DETAILED BUILDING CLASS TABLE
# =============================================================================

print("\n" + "="*80)
print("📋 DETAILED BUILDING CLASS BREAKDOWN")
print("="*80)

print(f"\n{'Code':<6} {'Building Type':<45} {'Count':>8} {'Median Price':>15} {'$/SqFt':>10}")
print("-" * 90)

class_stats = df.groupby('bldgclass').agg({
    'sale_price': ['count', 'median'],
    'price_per_sqft': 'median'
}).round(0)

class_stats.columns = ['Count', 'Median Price', 'Median PPS']
class_stats = class_stats.sort_values('Count', ascending=False)

for bldg_class, row in class_stats.head(25).iterrows():
    name = BUILDING_CLASS_NAMES.get(bldg_class, 'Unknown')[:43]
    pps = row['Median PPS'] if pd.notna(row['Median PPS']) else 0
    print(f"{bldg_class:<6} {name:<45} {int(row['Count']):>8,} ${row['Median Price']:>13,.0f} ${pps:>8,.0f}")

# =============================================================================
# 9. SAVE PROCESSED DATA
# =============================================================================

print("\n" + "="*80)
print("💾 SAVING PROCESSED DATA")
print("="*80)

# Save complete dataset
df.to_csv('nyc_housing_analyzed.csv', index=False)
print(f"  ✓ nyc_housing_analyzed.csv ({len(df):,} records)")

# Save residential only
df_residential = df[df['use_type'] == 'Residential'].copy()
df_residential.to_csv('nyc_housing_residential.csv', index=False)
print(f"  ✓ nyc_housing_residential.csv ({len(df_residential):,} records)")

# Save summary stats
building_stats.to_csv('nyc_building_stats.csv')
print(f"  ✓ nyc_building_stats.csv")

# =============================================================================
# FINAL SUMMARY
# =============================================================================

print("\n" + "="*80)
print("✅ ANALYSIS COMPLETE!")
print("="*80)

print(f"""
📊 Data Files Created:
  • nyc_housing_analyzed.csv - Full dataset with all classifications
  • nyc_housing_residential.csv - Residential properties only
  • nyc_building_stats.csv - Building category statistics

🗺️ Interactive Maps Created:
  • nyc_property_map.html - All properties by borough
  • nyc_property_map_by_type.html - Properties by building type
  • nyc_price_heatmap.html - Price per sqft density map

📈 Analytics Dashboards Created:
  • nyc_analytics_dashboard.html - Main analytics dashboard
  • nyc_building_types_by_borough.html - Building types breakdown
  • nyc_price_by_building_type.html - Price by building type
  • nyc_borough_analysis.html - Borough deep dive

📋 Summary:
  • Total Properties: {len(df):,}
  • Boroughs: {df['borough_name'].nunique()}
  • Building Categories: {df['building_category'].nunique()}
  • Unique Building Classes: {df['bldgclass'].nunique()}
  • Median Price: ${df['sale_price'].median():,.0f}
  • Median $/SqFt: ${df[df['price_per_sqft'] > 0]['price_per_sqft'].median():,.0f}

🎉 Open the HTML files in your browser to explore the interactive visualizations!
""")